In [1]:
import sparknlp
sparknlp.start(spark32=True)

22/04/11 10:27:09 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/11 10:27:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lbridges/.ivy2/cache
The jars for the packages stored in: /home/lbridges/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-spark32_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-08ace179-7681-45a6-9c5b-abb02d2a8329;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-spark32_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found or

In [2]:
from pyspark.sql import SparkSession
# start spark session configured for spark nlp
spark = SparkSession.builder \
     .master('local[*]') \
     .appName('Spark NLP') \
     .config('spark.jars.packages', 
             'com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.2') \
     .getOrCreate()

# from pyspark.sql import SparkSession

# spark = sparknlp.start(spark32=True)

In [3]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

In [4]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [5]:
documentAssembler = DocumentAssembler() \
     .setInputCol('value') \
     .setOutputCol('document')
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('token')
# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)
# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma')
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemma']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False) \
     .setStopWords(eng_stopwords)
# finisher converts tokens to human-readable output
finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[ / ]

[OK!]


In [6]:
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           lemmatizer,
           stopwords_cleaner,
           finisher
     ])

In [8]:
# import data

from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window, SQLContext
sqlContext = SQLContext(spark)


path = 'classic'
data = spark.read.text(path, wholetext=True)

books = ["erewhon", "time_traveler"]
b = sqlContext.createDataFrame([(l,) for l in books], ['Book'])
data = data.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

data = b.join(data, b.row_idx == data.row_idx).\
    drop("row_idx")


/opt/spark/python/lib/pyspark.zip/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.


In [9]:

# transform text with the pipeline
processed_data = pipeline.fit(data).transform(data)

In [10]:
processed_data.createOrReplaceTempView("processed")
spark.sql("Select * from processed").show()

22/04/11 10:30:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 10:30:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 10:30:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 10:30:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 10:30:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 10:30:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         Book|               value|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      erewhon|CHAPTER XXIII. TH...|[{document, 0, 56...|[{token, 0, 6, CH...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[chapter, xxiii, ...|
|time_traveler|The Time Travelle...|[{document, 0, 58...|[{token, 0, 2, Th...|[{token, 0, 2, th...|[{token, 0, 2, th...|[{token, 4, 7, ti...|[time, traveller,...|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [11]:
from pyspark.sql.functions import explode, col
scifi_words = processed_data.withColumn('exploded_text', 
                               explode(col('finished_clean_lemma')))

In [25]:
scifi_words.createOrReplaceTempView("scifi")
spark.sql("Select * from scifi").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|               value|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma| exploded_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|CHAPTER XXIII. TH...|[{document, 0, 56...|[{token, 0, 6, CH...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[chapter, xxiii, ...|       chapter|
|CHAPTER XXIII. TH...|[{document, 0, 56...|[{token, 0, 6, CH...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[chapter, xxiii, ...|         xxiii|
|CHAPTER XXIII. TH...|[{document, 0, 56...|[{token, 0, 6, CH...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[{token, 0, 6, ch...|[chapter, xxiii, ...|          book|
|CHAPTER XXIII. 

In [26]:
counts = scifi_words.groupby('exploded_text').count()
counts_pd = counts.toPandas()
scifi_dict = {counts_pd.loc[i, 'exploded_text']: 
                counts_pd.loc[i, 'count'] 
                for i in range(counts_pd.shape[0])}

22/04/11 11:01:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 11:01:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 11:01:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 11:01:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 11:01:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/11 11:01:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
counts.show()

In [27]:
counts_pd

,exploded_text,count
0,chapter,3
1,xxiii,1
2,book,1
3,machine,95
4,writer,11
...,...,...
1762,hard,1
1763,smile,1
1764,sure,1
1765,backward,1


In [28]:
scifi_dict

{'chapter': 3,
 'xxiii': 1,
 'book': 1,
 'machine': 95,
 'writer': 11,
 'commencesthere': 1,
 'time': 40,
 'earth': 9,
 'appearance': 2,
 'utterly': 2,
 'destitute': 1,
 'animal': 26,
 'vegetable': 8,
 'life': 33,
 'accord': 1,
 'opinion': 1,
 'good': 17,
 'philosopher': 1,
 'simply': 3,
 'hot': 1,
 'round': 1,
 'ball': 2,
 'crust': 1,
 'gradually': 2,
 'cool': 1,
 'human': 14,
 'exist': 16,
 'state': 6,
 'allow': 5,
 'see': 25,
 'though': 16,
 'world': 6,
 'concern': 2,
 'entirely': 4,
 'ignorant': 1,
 'physical': 3,
 'science': 3,
 'would': 34,
 'pronounce': 1,
 'impossible': 3,
 'creature': 9,
 'possess': 5,
 'anything': 10,
 'like': 14,
 'consciousness': 21,
 'evolve': 2,
 'seem': 13,
 'cinder': 1,
 'behold': 1,
 'deny': 4,
 'contain': 3,
 'potentiality': 2,
 'yet': 12,
 'course': 6,
 'come': 13,
 'possible': 8,
 'may': 34,
 'even': 20,
 'new': 13,
 'channel': 1,
 'dig': 3,
 'detect': 3,
 'sign': 1,
 'present': 30,
 'acceptation': 1,
 'term': 4,
 'thinga': 1,
 'thing': 30,
 'far': 

In [13]:
# Helper Functions

def term_frequency(BoW_dict):
     tot_words = sum(BoW_dict.values())
     freq_dict = {word: BoW_dict[word]/tot_words 
                  for word in BoW_dict.keys()}
     return freq_dict
     
from math import log
def inverse_document_frequency(list_of_dicts):
    tot_docs = len(list_of_dicts)
    words = set([w for w_dict in list_of_dicts 
                   for w in w_dict.keys()])
    idf_dict = {word: log(float(tot_docs)/
                      (1.0 + sum([1 for w_dict in list_of_dicts 
                              if word in w_dict.keys()]))) 
                    for word in words}
    return idf_dict

def tf_idf(list_of_dicts):
     words = set([w for w_dict in list_of_dicts 
                  for w in w_dict.keys()])
     tf_idf_dicts = []
     idfs = inverse_document_frequency(list_of_dicts)
     for i, w_dict in enumerate(list_of_dicts):
          w_dict.update({word: 0 for word in words 
                         if word not in w_dict.keys()})
          tf = term_frequency(w_dict)
          tf_idf_dicts.append({word: tf[word]*idfs[word] 
                               for word in words})
     return tf_idf_dicts

In [18]:
list_of_word_dicts = [company_complaint_word_counts_dict[company] 
                      for company in companies]
tf_idf_by_book_list = tf_idf(list_of_word_dicts)
tf_idf_by_book_dict = {c: tf_dict 
           for c, tf_dict in zip(companies, tf_idf_by_book)}

NameError: name 'companies' is not defined